# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = 'city_data.csv'

city_data = pd.read_csv(city_csv)

city_data




,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Kharp,100,RU,1579841085,96,66.80,65.81,7.72,1.86
1,1,Castro,89,BR,1579841017,99,-24.79,-50.01,57.60,3.71
2,2,Tasiilaq,100,GL,1579841028,61,65.61,-37.64,17.60,11.41
3,3,Punta Arenas,64,CL,1579841064,65,-53.15,-70.92,46.20,10.29
4,4,Mount Isa,40,AU,1579840931,83,-20.73,139.50,80.60,6.93
...,...,...,...,...,...,...,...,...,...,...
579,635,Koungou,93,YT,1579841314,88,-12.73,45.20,82.40,13.87
580,636,Curuguaty,8,PY,1579841314,96,-24.47,-55.69,64.31,2.01
581,637,Talbahat,0,IN,1579841315,27,25.05,78.43,66.54,8.32
582,638,Muros,100,ES,1579841160,100,42.78,-9.06,42.24,2.42


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = city_data[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

humidity_map = city_data['Humidity'].astype('float')

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig  

Figure(layout=FigureLayout(height='420px'))

[[{'geometry': {'location': {'lat': -35.7178714, 'lng': 150.1908089},
    'viewport': {'northeast': {'lat': -35.7165624697085,
      'lng': 150.1921486802915},
     'southwest': {'lat': -35.7192604302915, 'lng': 150.1894507197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
   'id': 'a1c92e6ce1323c6c50c137477fb799df737a847a',
   'name': 'Coachhouse Marina Resort - Batemans Bay',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 768,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100413788186485123048">Coachhouse Marina Resort - Batemans Bay</a>'],
     'photo_reference': 'CmRaAAAACOGkE-FRhRa_zX_eesV6-TfuAKnf1zG6nWspSXZrMh4WxmrZ_ktSQm_6gFfixC4UjPQYI_vRROiHsyclQbHx9Gs1ANP-jwDvad6j_l1K-uDMsR0SnBCtP80nXJT6f_tiEhDx2TF-knozHmqO_YnwkXL4GhRmpcN8gLiajQTQBzWMzW8Shrj3pg',
     'width': 1366}],
   'place_id': 'ChIJ8SSGGR_AFWsRcnqdXYFT6Rs',
   'plus_code': {'compound_code': '75JR+V8 Batemans Bay, New South Wales, Austra

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
city_ideal = city_data[(city_data["Max Temp"]> 70) & (city_data["Max Temp"] < 80) & (city_data["Wind Speed"] < 10) & (city_data["Cloudiness"] == 0)]

city_ideal

ideal_coords = []

for x in range(len(city_ideal)):
    ideal_coords.append(0)

y = 0

for x in city_ideal.index:
     ideal_coords[y] = str(city_ideal['Lat'][x]) + ',' + str(city_ideal['Lng'][x])
     y += 1   

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
json = []

for x in range(len(ideal_coords)):
    json.append(0)
   
ideal_coords = pd.DataFrame(ideal_coords)

for x in ideal_coords.index:
    params = {"location": ideal_coords[0][x],"radius": "5000",
            "type": "hotels",
            "key": g_key}

    response = requests.get(base_url, params=params).json()

    json[x] = response['results']

coord_list = {"Batemans Bay":[{"Name": "Empty","Lat":"Empty","Lng":"Empty"}],"Neuquen":[{"Name": "Empty","Lat":"Empty","Lng":"Empty"}],"Praia":[{"Name": "Empty","Lat":"Empty","Lng":"Empty"}],"Jiwani":[{"Name": "Empty","Lat":"Empty","Lng":"Empty"}],"Young":[{"Name": "Empty","Lat":"Empty","Lng":"Empty"}]}

city_hotel_list = {}

for x in range(len(city_ideal["City"])):
    city_hotel_list[x] = []


for y in range(len(city_hotel_list)):
     for z in range(len(json)):
         if z==y:
             for x in range(len(json[z])):
                 city_hotel_list[y].append([json[z][x]['name'],json[z][x]['geometry']['location']['lat'],json[z][x]['geometry']['location']['lng']])
       


In [7]:
city_hotel_list

hotel_df = []

for x in city_hotel_list:
    hotel_df.append(city_hotel_list[x][1])
    
hotel_df = pd.DataFrame(hotel_df)

hotel_df["City"] = [0,0,0,0,0]

hotel_df["City"] = [x for x in city_ideal["City"]]

hotel_df = hotel_df.rename(columns = {0:"Hotel Name",1:"Lat",2:"Lng"})

narrowed_city_df = city_ideal.merge(hotel_df, left_on='City', right_on='City')

narrowed_city_df = narrowed_city_df.rename(columns = {"Lat_x":"City_Lat","Lng_x":"City_Lng","Lat_y":"Lat","Lng_y":"Lng"})

narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,City_Lat,City_Lng,Max Temp,Wind Speed,Hotel Name,Lat,Lng
0,84,Batemans Bay,0,AU,1579840969,66,-35.72,150.18,78.71,1.99,Coachhouse Marina Resort - Batemans Bay,-35.717871,150.190809
1,166,Neuquen,0,AR,1579841114,29,-38.95,-68.06,77.00,4.70,Hotel del Comahue,-38.951079,-68.058805
2,230,Praia,0,CV,1579841169,46,14.92,-23.51,73.40,9.17,LT Aparthotel,14.904834,-23.526229
3,312,Jiwani,0,PK,1579841199,60,25.05,61.74,70.95,7.25,National Bank Of Pakistan,25.050696,61.747467
4,553,Young,0,UY,1579841285,67,-32.68,-57.63,73.99,1.01,Municipio de Young,-32.699790,-57.627570


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map

locations = city_data[["Lat", "Lng"]].astype(float)

fig = gmaps.figure()

humidity_map = city_data['Humidity'].astype('float')

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_map, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)



hotel_loc = hotel_df[["Lat", "Lng"]]

hotel_layer = gmaps.symbol_layer(hotel_loc, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=10,
    info_box_content=hotel_info)


fig = gmaps.figure(zoom_level=2, center=(0, 0))

fig.add_layer(heat_layer)

fig.add_layer(hotel_layer)

fig  

# Display Map

Figure(layout=FigureLayout(height='420px'))

['\n<dl>\n<dt>Name</dt><dd>Coachhouse Marina Resort - Batemans Bay</dd>\n<dt>City</dt><dd>Batemans Bay</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hotel del Comahue</dd>\n<dt>City</dt><dd>Neuquen</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>LT Aparthotel</dd>\n<dt>City</dt><dd>Praia</dd>\n<dt>Country</dt><dd>CV</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>National Bank Of Pakistan</dd>\n<dt>City</dt><dd>Jiwani</dd>\n<dt>Country</dt><dd>PK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Municipio de Young</dd>\n<dt>City</dt><dd>Young</dd>\n<dt>Country</dt><dd>UY</dd>\n</dl>\n']